In [1]:
# if you need to install Pybullet:
#   pip3 install Pybullet   (python3)
#   pip install PyBullet    (python2.x)

# let's start with imports
import numpy as np
from numpy.random import seed
from numpy.random import rand

from engine import *

# seed random number generator
seed(1)

pybullet build time: Nov 28 2023 23:51:11


In [2]:
# create a pybullet engine from the engine module
engine = PyBulletEnvironment()


In [3]:
#load shovel
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0,0,0])
robot_urdf = './urdf/shovel/shovelFlat.urdf'

# load the robot
engine.open_environment(robot_urdf, startPos, startOrientation)

MESA: warning: Driver does not support the 0xa7a0 PCI ID.
libGL error: failed to create dri screen
libGL error: failed to load driver: iris
MESA: warning: Driver does not support the 0xa7a0 PCI ID.
libGL error: failed to create dri screen
libGL error: failed to load driver: iris


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org


In [4]:
# load pebbles

# get shovel_link position
shovelLinkState = p.getLinkState(engine.ID[1], 2)

# admissible init positions
minpos = np.asarray(shovelLinkState[0]) - 5*1e-1
maxpos = np.asarray(shovelLinkState[0]) + 5*1e-1

# number of pebbles
pebbleNum = 0

# store pebbles IDs
pebbleId = np.zeros(pebbleNum)

# scatter pebbles
for i in range (pebbleNum):
    
    # generate init pos
    startPos = minpos + (rand(3) * (maxpos - minpos))
    startPos[-1] = 0.2
    
    # set orientation
    startOrientation = p.getQuaternionFromEuler([0,0,0])
    
    # load pebble 
    engine.load_urdf('./urdf/pebbles/pebbles.urdf', startPos, startOrientation)    
    
# simulate
engine.simulate(0.5)

In [5]:
# here we control the shovel to move to a target position and orientation using velocity control
# the loop controls the velocity of the joint until the target position is reached

# in the next params we define the control mode, the max force on the joint, the target position and velocity

# get the current end effector position
endEffectorPos = p.getLinkState(engine.ID[1], 2)[0]
pathLength = 1
pathCurvature = 5
Nwaypoints = 10

# compute the waypoints and print them
waypoints = engine.get_waypoints(endEffectorPos, pathLength, pathCurvature, Nwaypoints)
engine.draw_path(waypoints)
print('Initial position: ' + str(endEffectorPos))
print('Waypoints:\n' + '\n'.join(str(waypoint) for waypoint in waypoints))


Initial position: (0.7, 0.0, 0.025)
Waypoints:
[0.7000000000000003, 0.0, 0.025]
[0.811101966389493, 0.0012345170968073305, 0.025]
[0.922149069868188, 0.004937458774245229, 0.025]
[1.0330864746169652, 0.011106996494388177, 0.025]
[1.1438593989866903, 0.01974008369732072, 0.025]
[1.2544131425497653, 0.030832457305540295, 0.025]
[1.3646931131115705, 0.044378639829103506, 0.025]
[1.4746448536684689, 0.060371942070440276, 0.025]
[1.5842140692990456, 0.07880446642753025, 0.025]
[1.6933466539753061, 0.09966711079379209, 0.025]


In [6]:
# here we control

# target velocities - one for each joint
# this is only used in velocity control. In position control is a no-care parameter
targetVel = [0.1, 0.2, 0.2]

# control mode
if 0:
    # # velocity control
    mode = p.VELOCITY_CONTROL
    # max force on joint
    maxForce = [1e3, 1e3, 1e3]
else:
    # position control
    mode = p.POSITION_CONTROL
    # max force on joint
    maxForce = [1e3, 1e3, 1e3]

engine.control_waypoints(waypoints, targetVel, control_mode=mode, max_force=maxForce)

: 